<h2>KOSMOS slitmask reduction

To have a good idea of the functionalities of the notebook, the user is highly advised to read the workflow.
This notebook is assuming that the user has astro-pyvista (pyvista) installed. If not, visit (https://pyvista.readthedocs.io/en/latest/installation.html) for installation instructions.

# Necessary Modules

In [1]:
# If you are planning to run matplotlib qt, check if you have the following packages; if not, run this cell.
#!pip install scikit-image
#!pip install scikit-learn
#!pip install PyQt6
#!pip install PySide6
#!pip install PyQt5

In [2]:
from pyvista import imred, tv, stars, slitmask, image, spectra
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack
import pandas as pd
from astropy.io import fits

**Setup the TV window**

In [3]:
# Use these lines if you are running the notebook yourself. Matplotlib
# window will open outside the notebook, which is the desired behavior so
# you can have a single display tool, which you should leave open. Other
# plot windows will also appear outside the notebook, which you can close
# as desired
# you may need/want to use qt or osx in the next line
%matplotlib qt    
t=tv.TV()
# following lines only for fully non-interactive demo of notebook
#%matplotlib inline
#plotinter=False
#t=None

# Directory setup

In [4]:
# insert the directory name with images here
indir='UT230909'
red=imred.Reducer('KOSMOS',dir='UT230909')

INSTRUMENT: KOSMOS   config: 
  will use format:  UT230909/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    33    20  4056 
  2105    33    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [5]:
# Display the contente of the directory
red.log().show_in_notebook(display_length=5)

idx,FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
0,Flat_SEG3G2.0001.fits,2023-09-09T01:31:47.843412,,6:56:00.00,75:00:00.00,2.0
1,Flat_SEG3G2.0002.fits,2023-09-09T01:33:24.544267,,6:56:00.00,75:00:00.00,1.0
2,Flat_SEG3G2.0003.fits,2023-09-09T01:34:47.647398,,6:56:00.00,75:00:00.00,0.5
3,Flat_SEG3R2.0004.fits,2023-09-09T01:38:24.858222,,6:56:00.00,75:00:00.00,0.5
4,Flat_EMPTY_TEST.0005.fits,2023-09-09T01:40:21.135840,,6:56:00.00,75:00:00.00,0.5
5,Flat_SEG3R1.0006.fits,2023-09-09T01:43:41.792202,,6:56:00.00,75:00:00.00,0.5
6,Flat_SEG3R1.0008.fits,2023-09-09T01:47:35.037138,,6:56:00.00,75:00:00.00,0.5
7,Flat_SEG3G2.0009.fits,2023-09-09T01:49:32.885458,,6:56:00.00,75:00:00.00,0.5
8,SEG3G2.0010.fits,2023-09-09T02:19:53.351408,Seg3G2-Mask,21:21:32.11,19:08:59.20,20.0
9,SEG3G2.0011.fits,2023-09-09T02:33:52.513049,Seg3G2-Mask,21:21:34.14,19:09:06.69,20.0


# Calibration images.

## Master bias.

In [6]:
biastims=[74,75,76,77,78] # Make a list of biases using file numbers.
bias=red.mkbias(biastims,display=None) # Make a master bias.

  Reading file: UT230909\Bias.0074.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0075.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0076.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0077.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Bias.0078.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [7]:
# Display master bias.
if t is not None:
    t.tv(bias)

## Master dark

In [8]:
darktims=[94,95,96] # Make a list of darks using file numbers.
dark=red.mkdark(darktims,display=None, clip =5) # Make a master dark.

  Reading file: UT230909\dark.0094.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0095.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\dark.0096.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [9]:
# Display dark.
if t is not None:
    t.tv(dark)

## Master flat

In [10]:
flatims=[21,22] # Make a list of flats using file numbers.
flat=red.mkflat(flatims,spec=True) # Make masteer flat.

  Reading file: UT230909\Flat_SEG3G2.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: UT230909\Flat_SEG3G2.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


In [11]:
# Display flat
if t is not None:
    t.tv(flat)

## Master arcs

In [12]:
arcs= red.sum([23,24]) # Make a master arc

  Reading file: UT230909\Ne_120s_SEG3G2.0023.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  Reading file: UT230909\Ne_360s_SEG3G2.0024.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  combining data with sum....
  calculating uncertainty....


In [13]:
# Display master arcs.
if t is not None:
    t.tv(arcs)

Use flat1 for slit identification.

In [14]:
flat1 = red.reduce(21)

  Reading file: UT230909\Flat_SEG3G2.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


# Find and display slits on a flat image

Find slit edges from flat and fit polynomials to locations. Please do not use the master arc because the data format has the overscan stored in it.

In [15]:
trace0=spectra.Trace(transpose=True) # Setup a trace base based on KOMOS (Trancepose = True)
t.tvclear()
bottom,top = trace0.findslits(flat1,display=t,thresh=0.5,sn=True) # This does not take mkflat().

# Read slit mask from a kms-file

Read in the slitmask file, which we'll use to get object names and slit locations to help with wavelength solution.

If the number of slitlets found doesn't match the number of targets, you'll need to go back and adjust the threshold to find the correct slitlets, or else modify the targets table below so that they match.

In [16]:
kmsfile='kms/Copy of kosmos.23.seg3g2.kms' # File directory.
targets=slitmask.read_kms(kmsfile,sort='YMM') # Reading file.
if len(targets) != len(bottom) : # checking if the number of slits found matches the targets.
    print('ERROR, number of identified slits does not match number of targets')
targets # Visualing file content.

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.87,-10.801
TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.61
TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272


## Convert the targets table to a pandas Data Frame

In [17]:
df = targets.to_pandas() # Convertion taking place.
df # A look at your table in panda format for index number.

,ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
0,TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
1,TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
2,TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
3,TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.870,-10.801
4,TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.610
5,TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
6,TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
7,TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
8,TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272
9,TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.780


# The function for filtering stars of interest.

There isn't much to explain here; read the workflow or the function itself to grasp its functionalities.

In [18]:
def select_targets(targets, trace):
    """
    Function to select targets based on index, ID, or Name.

    Parameters:
    targets (pd.DataFrame or similar): The input DataFrame containing target data.
    trace (object): The trace object containing model and rows attributes.

    Returns:
    tuple: The updated trace object and the subset of selected targets DataFrame.
    """
    # Convert targets to a Pandas DataFrame if it isn't one already
    if not isinstance(targets, pd.DataFrame):
        data = targets.to_pandas()

    # Display the options to the user
    print("Select targets based on:")
    print("1. Index")
    print("2. ID")
    print("3. Name")
    choice = input("Enter the number corresponding to your choice: ")
    #data = targets.to_pandas()
    gd = []

    if choice == '1':
        print("You chose to select targets by index.")
        indices = input("Enter the indices (comma-separated): ")
        gd = [int(idx.strip()) for idx in indices.split(',')]
    
    elif choice == '2':
        print("You chose to select targets by ID.")
        ids = input("Enter the IDs (comma-separated): ")
        ids = [id.strip().strip("'\"") for id in ids.split(',')]
        # Ensure IDs are strings and strip any extra spaces
        data['ID'] = data['ID'].astype(str).str.strip()
        # Find the matching indices
        gd = data.index[data['ID'].isin(ids)].tolist()
    
    elif choice == '3':
        print("You chose to select targets by Name.")
        names = input("Enter the Names (comma-separated): ")
        names = [name.strip().strip("'\"") for name in names.split(',')]
        # Ensure names are stripped of extra spaces
        data['NAME'] = data['NAME'].astype(str).str.strip()
        # Find the matching indices
        gd = data.index[data['NAME'].isin(names)].tolist()
    
    else:
        print("Invalid choice. Defaulting to selection by Index.")

    if not gd:
        print("No valid selections were made. Returning the full DataFrame.")
        return trace, targets

    # Selecting the subset of targets based on the user's choice
    gdtrace = copy.deepcopy(trace)
    gdtrace.model = [trace.model[i] for i in gd]
    gdtrace.rows = [trace.rows[i] for i in gd]
    trace = copy.deepcopy(gdtrace)
    selected_targets = targets[gd]

    return trace, selected_targets

**Proceed with selection.**

In [19]:
# Samples to use for trail run. 
# indexes: 6,7,9,11,13, 
# ID: TARG108,TARG107,TARG106,TARG104,TARG102
trace, targets = select_targets(targets, trace0) 

Select targets based on:
1. Index
2. ID
3. Name


Enter the number corresponding to your choice:  2


You chose to select targets by ID.


Enter the IDs (comma-separated):  TARG108,TARG107,TARG106,TARG104,TARG102


In [20]:
vars(trace) # filtered trace

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([1089.05958351,    6.49312832,    5.26758998], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1154.49886865,    6.51239823,    6.81155863], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1352.12850914,    6.53479617,   11.59481407], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1542.12042917,    6.63076955,   16.0577897 ], domain=[  98., 3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([1680.82160588,    6.6160561 ,   19.38130477], domain=[  98., 3998.], window=[-1.,  1.], symbol='x')],
 'sigmodel': None,
 'sc0': None,
 'rows': [[1089, 1128],
  [1154, 1193],
  [1352, 1391],
  [1542, 1581],
  [1681, 1720]]}

In [21]:
targets #filtered targets

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG106,NN,STRAIGHT,0.9,10.0,0.0,212133.934,190720.87,0.154,1.707,-4.422,16.78
TARG104,NN,STRAIGHT,0.9,10.0,0.0,212136.755,190631.54,0.154,1.707,-11.247,25.194
TARG102,NN,STRAIGHT,0.9,10.0,0.0,212134.103,190555.71,0.154,1.707,-4.831,31.313


# Wavelength calibration

Using the derived traces, extract the slitlets for arcs

In [22]:
arcec=trace.extract2d(arcs,display=t)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....


Add XMM and YMM for each slit to headers of each extracted image. They need to match so you get the right values!

In [23]:
for arc,target in zip(arcec,targets) : 
    arc.header['XMM'] = target['XMM']
    arc.header['YMM'] = target['YMM']

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change). 

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier if you use an estimate of the shift from the mask design XMM.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D wavelength calibration.

You can really help this process if you supply an initial wavelength calibration (a pyvista WaveCal object) that was done using the same lamp(s) as your arc exposures (here, using KOSMOS_red_waves.fits'), and using a master line list that corresponds to these lamp(s) (here, using ne.dat). If you choose another WaveCal to start from, you may need to get a correct approximate relation for the shift from the reference spectrum as a function of XMM. For KOSMOS, seems like -22.5(XMM) gives a rough pixel shift from a center slit location, -22.5(XMM-24.44) for a low slit location.

The next cell just shows how the simple shift fails, even using the XMM for each slit to shift: all of the arc lines don't match up with just a translation.

In [24]:
plt.figure()
wav2=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
plt.plot(wav2.spectrum[0])
for i, arc in enumerate(arcec[0:1]) :
    shift=(arc.header['XMM']*-22.5)
    plt.plot(arc.data[19][int(shift):]*30)
    print(shift)
    plt.draw()

  rms:    0.177 Angstroms (50 lines)
-250.1775


The wavelength calibration for each slitlet will be saved. Since this is probably the most time-consuming part, you could use saved one if you only want to redo a couple of them (sometimes, you hit the wrong key and want to do one over!)

In [25]:
clobber=False  # set to False if you want to use any saved ones
for i,(arc,targ) in enumerate(zip(arcec,targets)) :
    wavname = 'CofIwav_{:s}.fits'.format(targ['ID'])
    if clobber or not os.path.exists(wavname) :
        wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
        wav.fit(degree=3)
        nrow=arc.shape[0]
    
        # get initial guess at shift from reference using XMM (KOSMOD red low!)
        shift=int(arc.header['XMM']*-22.5) # +550  #-wav.pix0)
        lags=np.arange(shift-20,shift+20)

        iter = True
        while iter :
            iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,
                                lags=lags,thresh=10,
                                file='new_wave_lamp/copy_new_neon_red_center.dat')
            lags=np.arange(-50,50)
            plt.close()

        bd= np.where(wav.weights<0.5)[0]
        print(wav.waves[bd])
        # Do the 2D wavelength solution, sampling 10 locations across slitlet
        wav.degree=5
        wav.identify(arc,plot=True,nskip=nrow//10,thresh=10)
        plt.close()
        wav.write(wavname)
        wav.add_wave(arc)
        t.tv(wav.correct(arc,arc.wave[nrow//2]))

Now set up routine to reduct/extract science frames

# Science image reduction

Science reduction with all calibration images applied.

In [26]:
star1=red.reduce(20, crbox='lacosmic', bias=bias, flat=flat, dark=dark, display=t)

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 
  subtracting bias...
  subtracting dark...
  starting CR rejection, may take some time ....
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
  flat fielding...


C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\imred.py:808: RuntimeWarning: divide by zero encountered in divide
  corr.data /= flat.data
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\imred.py:808: RuntimeWarning: invalid value encountered in divide
  corr.data /= flat.data
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\imred.py:809: RuntimeWarning: divide by zero encountered in divide
  corr.uncertainty.array /= flat.data


  See flat-fielded image and original with - (minus) key.
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Basic reduction with only cosmic ray ejection is applied.

In [27]:
# basic image reduction with cosmic rays
imcr=red.reduce(20,crbox='lacosmic',crsig=10,display=t)

  Reading file: UT230909\SEG3G2.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  See bias box (solid outlines applied to dashed regions of the same color), and cross section. 
   To continue, hit space in display window (p for debug) 
  starting CR rejection, may take some time ....
  zapping CRs with astroscrappy detect_cosmics
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


# Extraction

## 2D Extraction

2D extraction function. To understand what the function does, you can read through the workflow or the function itself.
If the user just wants to extract, run the function and follow the cell prompt instructions.

In [28]:
import matplotlib.pyplot as plt

def multi_extract2d(red, trace, targets, imcr, thresh=10, display=None, rad=5):
    # Helper function to process each slitlet, with optional wavelength adjustment
    def process_slitlet(o, targ, adjust_wavelength, display, rad):
        wav = spectra.WaveCal(f'./CofIwav_{targ["ID"]}.fits')
        orig = wav.model.c0_0
        
        wav.add_wave(o)  # Apply the wavelength model to the extracted slitlet
        
        if adjust_wavelength:
            # Perform skyline-based wavelength correction if adjustment is selected
            nrows = o.shape[0]
            rows = [x for x in range(nrows) if abs(x - nrows // 2) > rad]
            wav.skyline(o, thresh=thresh, rows=rows, plot=plot_enabled(display))
            wav.add_wave(o)  # Re-apply the wavelength model after correction

            if display:
                display.tv(o)  # Display the corrected 2D spectrum

            o = wav.correct(o, o.wave[nrows // 2])  # Correct wavelength using the central row

            if display:
                display.tv(o)  # Display the final corrected spectrum

            name = o.header["FILE"].split(".")[0]
            o.write(f'{name}_{targ["ID"]}_2d.fits')
            return wav.model.c0_0 - orig  # Return wavelength shift after correction
        else:
            # No wavelength adjustment, just correct and save the output
            nrows = o.shape[0]
            o = wav.correct(o, o.wave[nrows // 2])
            name = o.header["FILE"].split(".")[0]
            o.write(f'{name}_{targ["ID"]}_not_adjusted_2d.fits')
            plt.figure() 
            # Plot the center row of the slitlet
            plt.plot(o.wave[19], o.data[19], label=f'Approximated look for spec {i} with sky')
            plt.title('A possible extraction visualization ')
            plt.legend(loc = 'upper right')
            return None

    # Helper function to check if plotting is enabled
    def plot_enabled(display):
        if display is not None:
            display.clear()  # Clear display if provided
            return True  # Enable plotting
        return False

    # Perform 2D extraction
    out = trace.extract2d(imcr, display=display)

    # Ask the user if they want to perform wavelength adjustment
    print("Select a choice of wavelength adjustment:")
    print('1. 2D wavelength adjustment.')
    print('2. No 2D wavelength adjustment.')
    choice = input('Enter the number corresponding to your choice: ')
    
    adjust_wavelength = choice == '1'
    diffs = []

    # Loop over each slitlet and process it
    for i, (o, targ) in enumerate(zip(out, targets)):
        diff = process_slitlet(o, targ, adjust_wavelength, display, rad)
        if diff is not None:
            diffs.append(diff)

    if adjust_wavelength:
        # Print the wavelength shifts after adjustment
        print('Wavelength shifts:', diffs)
    
    return out  # Return the processed 2D spectra

Run the cell below to 2D extract.

In [29]:
out=multi_extract2d(red,trace,targets,imcr,display=t)

extracting: 
 1089-1128
 1154-1193
 1352-1391
 1542-1581
 1681-1720
  See extraction window(s). Hit space bar to continue....
Select a choice of wavelength adjustment:
1. 2D wavelength adjustment.
2. No 2D wavelength adjustment.


Enter the number corresponding to your choice:  1


  rms:    0.052
rejecting 13 points from 494 total: 
  rms:    0.052
rejecting 13 points from 494 total: 
  See identified lines.
  rms:    0.109
rejecting 2 points from 188 total: 
  rms:    0.103
rejecting 2 points from 188 total: 
  See 2D wavecal fit. Enter space in plot window to continue

appending uncertainty
appending bitmask
appending wave
  rms:    0.054
rejecting 14 points from 490 total: 
  rms:    0.054
rejecting 14 points from 490 total: 
  See identified lines.
  rms:    0.085
rejecting 0 points from 159 total: 
  See 2D wavecal fit. Enter space in plot window to continue

appending uncertainty
appending bitmask
appending wave
  rms:    0.041
rejecting 13 points from 493 total: 
  rms:    0.041
rejecting 13 points from 493 total: 
  See identified lines.
  rms:    0.091
rejecting 3 points from 158 total: 
  rms:    0.077
rejecting 3 points from 158 total: 
  See 2D wavecal fit. Enter space in plot window to continue

appending uncertainty
appending bitmask
appending wave

C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\spectra.py:708: OptimizeWarning: Covariance of the parameters could not be estimated
  coeff, var_matrix = curve_fit(gauss, xx, yy, p0=p0)


  See identified lines.
  rms:   23.285
rejecting 1 points from 191 total: 
  rms:    0.448
rejecting 1 points from 191 total: 
  See 2D wavecal fit. Enter space in plot window to continue

appending uncertainty
appending bitmask
appending wave
Wavelength shifts: [-0.07989621963133686, -0.04463752593346726, -0.057007243753105286, -0.07231412741930399, -0.052267084198319935]


## 1D Extraction

Like 2D extraction, 1D extraction has much happening in it. You should read the workflow to understand it. But if the user wishes just to extract, then run the cell and follow the prompt and instructions.  

In [31]:
# Main function for extracting 1D spectra from 2D data with optional sky adjustment and calibration
def multi_extract1d(spec2d, thresh_sky=12, linear=True, sky_cal=False, display=t,
                    skip= 10, degree=3, sigdegree=3, thresh=10,back_percentile=10,method='linear', 
                    file='pyvista/python/pyvista/data/sky/skyline.dat',plot=True):
    
    # Nested function to define a simple linear model for peak fitting
    def model(x, peak_value=0):
        return x * 0. + peak_value  # Returns a constant model function with the given peak_value
    
    # Function to handle the spectrum extraction, prompting the user for input
    def extract_spectrum(trace, spec2d_slice, i, rad_default=5):
        satisfied = False  # A flag to control user satisfaction with the extraction result
        while not satisfied:
            # Ask the user for the extraction radius, with a default value
            rad = int(input(f"Enter the radius for extraction for slit {i} (current default is {rad_default}): "))
            sky_width = rad - 5  # Compute the sky region width for background subtraction
            # Perform the spectrum extraction based on the radius and sky width
            spec = trace.extract(spec2d_slice, rad=rad, back=[[-10 + (-1 * sky_width), -rad], [10 + sky_width, rad]], display=display)
            # Ask the user if they are satisfied with the result
            user_response = input("Are you satisfied with the results? Enter 'y' for yes, anything else for no: ").strip().lower()
            if user_response == 'y':  # If user is satisfied, exit the loop
                satisfied = True
        return spec, rad  # Return the extracted spectrum

    # Function to process the extracted spectrum by saving it and plotting the data and sky
    def process_spectrum(spec, spec2d_slice, peak, i, prefix, rad):
        spec.wave = spec2d_slice.wave[peak]  # Assign the wavelength solution to the spectrum
        # Save the processed spectrum to a file, formatting the filename
        spec.write(f"{prefix}_adjust_rad_{rad}_{i:04d}.fits", overwrite=True)
        print(spec.wave[0].shape, spec.data[0].shape)  # Print the shape of the spectrum and wavelength arrays
        # Plot the spectrum and its sky background
        plt.figure()
        plt.subplot(2, 1, 1)
        plt.plot(spec.wave[0], spec.data[0], label=f'spec {i}')  # Plot the spectrum data
        plt.title('1D extracted spectrum')
        plt.legend(loc="upper right")
        plt.subplot(2, 1, 2)
        plt.plot(spec.wave[0], spec.sky[0], label=f'sky_spec {i}')  # Plot the sky background
        plt.title('1D sky spectrum')
        plt.legend(loc="upper right")
        plt.tight_layout()  # Adjust the plot layout to prevent overlapping elements
        return spec  # Return the processed spectrum

    # Function to find the peak and handle the spectrum extraction and processing
    def handle_peaks(trace, spec2d_slice, i, prefix, skip=skip):
        center = spec2d_slice.shape[0] // 2 # Setting width value.
        # Find the peak in the 2D spectrum using a threshold and width for peak detection
        peak, ind = trace.findpeak(spec2d_slice, thresh=thresh, width=center, sort=True, 
                                   back_percentile=back_percentile,method=method)
        if len(peak) > 0:  # If a peak is found
            srow = [peak[0]]  # Define the starting row based on the peak
            # Trace the spectrum along the identified peak
            trace.trace(spec2d_slice, srow, skip=skip, gaussian=True, display=display, rad=5)
            # Set the trace model using the peak position
            trace.model = [lambda x: model(x, peak_value=peak[0])]
            # Extract and process the spectrum
            spec, rad = extract_spectrum(trace, spec2d_slice, i)
            return process_spectrum(spec,spec2d_slice, peak, i, prefix,rad)
        else:
            print(f'No peak found for slit: {i}')  # If no peak is found, print a message
            return None  # Return None if no spectrum is extracted

    # Main function to adjust the spectrum, handling either 1D or 2D extraction based on user input
    def adjust_spectrum(spec2d, sky_check=False, degree=degree, sigdegree=sigdegree):
        spec1d = []  # Initialize an empty list to store the 1D spectra
        # Iterate through each 2D spectrum slice in the input data
        for i, spec2d_slice in enumerate(spec2d):
            # Initialize the trace object with default parameters for peak tracing
            trace1 = spectra.Trace(transpose=False, lags=range(-39, 39), sc0=None, degree=degree, sigdegree=sigdegree)
            trace1.rows = [0, spec2d_slice.data.shape[0]]  # Set the rows for tracing
            trace1.index = [0]  # Set the index for tracing
            # Define the filename prefix based on the user choice (1D or 2D)
            prefix = 'spec_2d' if choice == '1' else 'spec_1d'

            # Handle the peak finding, spectrum extraction, and processing
            spectrum = handle_peaks(trace1, spec2d_slice, i, prefix, skip)
            if spectrum:  # If the spectrum is successfully extracted
                if choice == '2' and sky_cal:  # If sky calibration is selected by the user
                    wav = spectra.WaveCal(f'./CofIwav_{targ["ID"]}.fits')  # Load the wavelength calibration file
                    swav = copy.deepcopy(wav)  # Create a deep copy of the wavelength calibration
                    # Perform sky line calibration on the spectrum
                    swav.skyline(spectrum, thresh=thresh_sky, linear=linear, plot=plot, rows=None, file=file)
                spec1d.append(spectrum)  # Add the processed spectrum to the list
        return spec1d  # Return the list of 1D spectra

    # Prompt the user for a choice between 1D and 2D adjustment
    print('If you did 2D adjustment select 1 else select 2:')
    print('1. No sky adjustment.')
    print('2. Sky adjustment.')
    choice = input('Enter the number corresponding to your choice: ')  # Get the user's choice as input

    # Adjust the spectrum based on the user's input and return the results
    return adjust_spectrum(spec2d, sky_cal)

Run the cell bellow to conduct 1D extraction.

In [32]:
spec1d = multi_extract1d(out,)

If you did 2D adjustment select 1 else select 2:
1. No sky adjustment.
2. Sky adjustment.


Enter the number corresponding to your choice:  1


looking for peaks using 38 pixels around 2048, threshhold of 10.000000
peaks:  [20]
aperture/fiber:  [0]


C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\spectra.py:1224: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
C:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyvista\spectra.py:1229: RuntimeWarning: invalid value encountered in divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<rad))[0]



  See trace. Hit space bar to continue....


...

appending uncertainty
appending bitmask
appending wave
appending sky
appending skyerr
(4096,) (4096,)
looking for peaks using 38 pixels around 2048, threshhold of 10.000000
peaks:  [16, 29, 35]
aperture/fiber:  [0, 1, 2]
  Tracing row: 16
  See trace. Hit space bar to continue....


Enter the radius for extraction for slit 4 (current default is 5):  4


  extracting ... 
  See extraction window(s). Hit space bar to continue....



Are you satisfied with the results? Enter 'y' for yes, anything else for no:  y


appending uncertainty
appending bitmask
appending wave
appending sky
appending skyerr
(4096,) (4096,)


# Checking the accuracy of wavelength calibration

In [33]:
f2_1 = fits.open('spec_2d_adjust_rad_4_0000.fits')
skyline_file_path = 'pyvista/python/pyvista/data/sky/skyline.dat'
skyline_wavelengths = np.loadtxt(skyline_file_path)
f2_1.info()

Filename: spec_2d_adjust_rad_4_0000.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      68   ()      
  1                1 ImageHDU         7   (4096, 1)   float64   
  2  UNCERT        1 ImageHDU         9   (4096, 1)   float64   
  3  BITMASK       1 ImageHDU        10   (4096, 1)   int32 (rescales to uint32)   
  4  WAVE          1 ImageHDU         8   (4096, 1)   float64   
  5  SKY           1 ImageHDU         8   (4096, 1)   float64   
  6  SKYERR        1 ImageHDU         8   (4096, 1)   float64   


**General checking plot.**

In [34]:
plt.figure()
plt.plot(f2_1[4].data[0], f2_1[6].data[0], )
for wl in skyline_wavelengths:
    plt.axvline(wl, color='red', linestyle='--', alpha=0.5)
plt.title('Sky Spectrum and emission lines Correspondence')
plt.grid(True)
plt.legend()

C:\Users\abdul\AppData\Local\Temp\ipykernel_23460\1453236354.py:7: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


**Zoomed in checking**

In [35]:
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.plot(f2_1[4].data[0], f2_1[6].data[0],label='range: 6350-7100')
ax2.plot(f2_1[4].data[0], f2_1[6].data[0],label='range: 7100-8533')
for wl in skyline_wavelengths:
    ax1.axvline(wl, color='red', linestyle='--', alpha=0.5)
    ax2.axvline(wl, color='red', linestyle='--', alpha=0.5)
ax1.set_ylabel('Intensity')
ax2.set_ylabel('Intensity')
ax1.set_xlabel('Angstrom')
ax2.set_xlabel('Angstrom')
ax1.set_title('Sky Spectrum and emission lines Correspondence')
ax2.set_title('Sky Spectrum and emission lines Correspondence')
plt.grid(True)
ax1.set_xlim(6350,7100)
ax2.set_xlim(7100,8533)
ax1.legend()
ax2.legend()

# Spectra stacking

**If the user have reduced at least two exposures of the same star then continue if not stop.**

In [ ]:
f02 = fits.open('spec_without_sky_17_004.fits')
f03 = fits.open('spec_without_sky_18_004.fits')
f03.info()

In [ ]:
from astropy.nddata import CCDData # Import astropy data class format. 
from ccdproc import Combiner # Import ccdproc combiner for stacking.

In [ ]:
#make a ccddata object out of the flux (pixel count) and uncertainty of each spectra you want to combine
ccd_67 = CCDData(data = f02[1].data[0], uncertainty = f02[2].data[0], unit = 'pixel')
ccd_68 = CCDData(data = f03[1].data[0], uncertainty = f03[2].data[0], unit = 'pixel')
#put them all into Combiner (from ccdproc)
spec_67_69 = Combiner([ccd_67, ccd_68])

#then average combine (it is slightly better than median combining)
avg_test_67_69 = spec_67_69.average_combine()

# Plot and compare.
plt.figure()

plt.plot(f02[4].data[0],f02[1].data[0], label= 'SEG3G2_17')
plt.plot(f03[4].data[0],f03[1].data[0], label= 'SEG3G2_18')
plt.plot(f02[4].data[0], avg_test_67_69.data, label = 'Stacked_spectrum')
plt.legend()
plt.xlabel('Angstrom')
plt.ylabel('Pixel count')

## Save the stacked spectra.

In [ ]:
hdu = fits.PrimaryHDU(avg_test_67_69)
hdul = fits.HDUList([hdu])

# Write to FITS file
hdul.writeto('Stacked_spec.fits', overwrite=True)